# Running a simulation

In [1]:
import pybamm

In [2]:
model = pybamm.lithium_ion.DFN()

In [3]:
sim = pybamm.Simulation(model)

In [4]:
sim.solve()

You can create a slider plot of the results. Note that in version 0.2.2 there is a bug that means variables which depend of space do not update with the slider in notebooks when the solution time is displayed in hours. This is fixed in version 0.2.2.post2

In [6]:
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

You can pass a list of variables to the plot command. To see all the variables in a model, type

In [7]:
model.variables

{'Time': Time(0x2712b4c4bbcf9f76, time, children=[], domain=[], auxiliary_domains={}),
 'Time [s]': Multiplication(-0x18e841428f0a62ad, *, children=['time', '96485.33212 * Maximum concentration in negative electrode [mol.m-3] * Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m] / function (absolute)'], domain=[], auxiliary_domains={}),
 'Time [min]': Division(0x6f248c3264e2545c, /, children=['time * 96485.33212 * Maximum concentration in negative electrode [mol.m-3] * Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m] / function (absolute)', '60.0'], domain=[], auxiliary_domains={}),
 'Time [h]': Division(0x6da68657aa79c39, /, children=['time * 96485.33212 * Maximum concentration in negative electrode [mol.m-3] * Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m] / function (absolute)', '3600.0'], domain=[], auxiliary_domains={}),
 'x': SpatialVariable(0x7da2

This is long! You can also search for variables 

In [8]:
model.variables.search("electrolyte")

Electrolyte concentration
Electrolyte concentration [Molar]
Electrolyte concentration [mol.m-3]
Electrolyte current density
Electrolyte current density [A.m-2]
Electrolyte flux
Electrolyte flux [mol.m-2.s-1]
Electrolyte potential
Electrolyte potential [V]
Electrolyte tortuosity
Gradient of electrolyte potential
Gradient of negative electrolyte potential
Gradient of positive electrolyte potential
Gradient of separator electrolyte potential
Negative electrolyte concentration
Negative electrolyte concentration [Molar]
Negative electrolyte concentration [mol.m-3]
Negative electrolyte potential
Negative electrolyte potential [V]
Negative electrolyte tortuosity
Positive electrolyte concentration
Positive electrolyte concentration [Molar]
Positive electrolyte concentration [mol.m-3]
Positive electrolyte potential
Positive electrolyte potential [V]
Positive electrolyte tortuosity
Separator electrolyte concentration
Separator electrolyte concentration [Molar]
Separator electrolyte concentration

Here we plot the electrode current density and terminal voltage

In [9]:
sim.plot(["Electrode current density", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

You can plot multiple variables on the same plot by nesting lists

In [10]:
sim.plot([["Electrode current density", "Electrolyte current density"], "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Options

You can pass options to each of the models. To see the available options look at the documentation for `BaseBatteryModel`. Here we set up a model in which the diffusion in the particles is assumed to be infinitely fast 

In [11]:
model_fast = pybamm.lithium_ion.DFN({"particle": "fast diffusion"}, name="fast DFN")

In [12]:
sim_fast = pybamm.Simulation(model_fast)

In [13]:
sim_fast.solve()

In [14]:
sim_fast.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

You can compare the output of two simulations

In [17]:
# Here we change the time unit to seconds so that the variables 
# update properly with the slider. The method `sim.plot()` automatically 
# picks a sensible time unit for you, so we couldn't use this workaround above
pybamm.dynamic_plot([sim, sim_fast], time_unit="seconds")

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=36.0), Output()), _dom_classes=…

and also pass variables to compare 

In [18]:
pybamm.dynamic_plot([sim, sim_fast], ["Terminal voltage [V]"], time_unit="seconds")

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=36.0), Output()), _dom_classes=…

## Accessing solution variables 

You can also access the solved variables directly so you can create your own plots

In [19]:
solution = sim.solution  # extract solution object

In [20]:
solution["Time [s]"]  # creates a post-processed variable

In [21]:
solution["Time [s]"].entries

array([   0.        ,   36.36363636,   72.72727273,  109.09090909,
        145.45454545,  181.81818182,  218.18181818,  254.54545455,
        290.90909091,  327.27272727,  363.63636364,  400.        ,
        436.36363636,  472.72727273,  509.09090909,  545.45454545,
        581.81818182,  618.18181818,  654.54545455,  690.90909091,
        727.27272727,  763.63636364,  800.        ,  836.36363636,
        872.72727273,  909.09090909,  945.45454545,  981.81818182,
       1018.18181818, 1054.54545455, 1090.90909091, 1127.27272727,
       1163.63636364, 1200.        , 1236.36363636, 1272.72727273,
       1309.09090909, 1345.45454545, 1381.81818182, 1418.18181818,
       1454.54545455, 1490.90909091, 1527.27272727, 1563.63636364,
       1600.        , 1636.36363636, 1672.72727273, 1709.09090909,
       1745.45454545, 1781.81818182, 1818.18181818, 1854.54545455,
       1890.90909091, 1927.27272727, 1963.63636364, 2000.        ,
       2036.36363636, 2072.72727273, 2109.09090909, 2145.45454

Can create a post-processed V that can be called at any time (by interpolation)

In [22]:
V = solution["Terminal voltage [V]"]

In [23]:
V([200, 400, 780, 1236])  # times in seconds

array([3.73210958, 3.71116961, 3.67920812, 3.65043568])

## Saving the simulation and data

You can save the whole simulation

In [24]:
model = pybamm.lithium_ion.SPM()

In [25]:
sim = pybamm.Simulation(model)

In [26]:
sim.solve()

In [27]:
sim.save("SPM.pkl")

In [28]:
sim2 = pybamm.load("SPM.pkl")

In [29]:
sim2.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

or just the solution

In [30]:
sol = sim.solution

In [31]:
sol.save("SPM_sol.pkl")

In [32]:
sol2 = pybamm.load("SPM_sol.pkl")

In [33]:
pybamm.dynamic_plot(sol2, time_unit="seconds")

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=36.0), Output()), _dom_classes=…

or just the data for some variables

In [34]:
sol.save_data("sol_data.pkl", ["Current [A]", "Terminal voltage [V]"])

can also save to csv or mat

In [35]:
sol.save_data("sol_data.csv", ["Current [A]", "Terminal voltage [V]"], to_format="csv")

In [36]:
sol.save_data("sol_data.mat", ["Current [A]", "Terminal voltage [V]"], to_format="matlab")